## Chương trình tạo thơ bằng mô hình transformer

In [ ]:
%%bash
# Ubuntu hiện không phân phối Chromium ngoài snap, vì vậy ta thêm repository Debian Buster
# Tham khảo: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Thêm repository Debian Buster
cat << "EOF" | sudo tee /etc/apt/sources.list.d/debian.list
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Thêm khóa công khai cần thiết từ keyserver của Ubuntu
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# Xuất các khóa và lưu vào keyrings tương ứng
sudo apt-key export 77E11517 | sudo gpg --dearmor -o /usr/share/keyrings/debian-buster.gpg
sudo apt-key export 22F3D138 | sudo gpg --dearmor -o /usr/share/keyrings/debian-buster-updates.gpg
sudo apt-key export E562B32A | sudo gpg --dearmor -o /usr/share/keyrings/debian-security-buster.gpg

# Ưu tiên repository Debian cho các gói liên quan đến Chromium
cat << "EOF" | sudo tee /etc/apt/preferences.d/chromium.pref
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Cập nhật danh sách gói và cài đặt Chromium cùng Chromium Driver
sudo apt-get update
sudo apt-get install -y chromium chromium-driver

# Cài đặt thư viện Selenium cho Python
pip install selenium


deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
Executing: /tmp/apt-key-gpghome.tkJdP6GNKD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
Executing: /tmp/apt-key-gpghome.Uz7fFQ9TXo/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
Executing: /tmp/apt-key-gpghome.8nFNMCMNJw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [5

gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 27.)
debconf: falling back to frontend: Rea

In [ ]:
# import các thư viện cần thiết
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



với selenium, ta có thể hiểu đơn giản rằng việc truy cập vào một trang web sẽ được thực hiện như chính chúng ta sử dụng trình duyệt wev hàng ngày. Đầu tiênta khởi tạo một driver:

In [ ]:
WEBDRIVER_DELAY_TIME_INT = 10
TIMEOUT_INT = 10
service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(TIMEOUT_INT)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)


In [ ]:
# đưa vào một hàm để bóc tách các trang web trong thơ
def extract_poem_links(driver, page_idx):
    main_url = f"https://www.thivien.net/searchpoem.php?PoemType=16&Page={page_idx}"
    driver.get(main_url)
    time.sleep(random.uniform(3, 5))

    content_tags_xpath = '//*[ @class ="page-content container"]//div[@class="page-content-main"]//div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    poem_links = []

    for tag in content_tags:
        try:
            link_element = tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            poem_title = link_element.text
            poem_url = link_element.get_attribute("href")
            poem_links.append({"title": poem_title, "url": poem_url})
        except Exception as e:
            print(f"Error extracting link: {e}")
            continue

    return poem_links


In [ ]:
link = extract_poem_links(driver, 10)

In [ ]:
link

[{'title': 'Ai chờ ai mùa thu',
  'url': 'https://www.thivien.net/Cao-Qu%E1%BA%A3ng-V%C4%83n/Ai-ch%E1%BB%9D-ai-m%C3%B9a-thu/poem-1EbqmZ89BXfz2sGXzcG_8w'},
 {'title': 'Ai cười mà xinh thế',
  'url': 'https://www.thivien.net/Nguy%E1%BB%85n-Th%E1%BA%BF-Ho%C3%A0ng-Linh/Ai-c%C6%B0%E1%BB%9Di-m%C3%A0-xinh-th%E1%BA%BF/poem-JsXBDw73b9M2DOh9g0SNXA'},
 {'title': 'Ai đón hạ',
  'url': 'https://www.thivien.net/Thu-Phong/Ai-%C4%91%C3%B3n-h%E1%BA%A1/poem-afFLrXZR-pcnB-M2ojGukw'},
 {'title': 'Ai người hiểu rõ nhất?',
  'url': 'https://www.thivien.net/Ph%C3%B9ng-Qu%C3%A1n/Ai-ng%C6%B0%E1%BB%9Di-hi%E1%BB%83u-r%C3%B5-nh%E1%BA%A5t/poem-4Ozs5Ws2Y640ifsWTQiW8Q'},
 {'title': 'Ai níu mùa xuân đấy',
  'url': 'https://www.thivien.net/L%C3%AA-Th%E1%BB%8B-Kim/Ai-n%C3%ADu-m%C3%B9a-xu%C3%A2n-%C4%91%E1%BA%A5y/poem-mHKUylp0Ay4_lEHEyPiDHw'},
 {'title': 'Ai về khuya',
  'url': 'https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%BA%A3ng-H%C3%A0n/Ai-v%E1%BB%81-khuya/poem-Ryze7jz3QInxVqCI5EnYBg'},
 {'title': 'Ám ảnh',
  'url': 

In [ ]:
import re
def clean_poem_html(html):
    html = re.sub(r"<img.*?>", "", html, flags=re.IGNORECASE | re.DOTALL) # xóa các thẻ ảnh
    html = re.sub(r"<i>.*?</i>", "", html, flags=re.IGNORECASE | re.DOTALL) # loại bỏ các thẻ in nghiêng
    html = re.sub(r"<b>(.*?)</b>(?!\s*(?:<br\s*/?>\s*){2,})",
                  r"\1",
                  html,
                  flags=re.IGNORECASE) # loại bỏ các thẻ in đậm nhưng giữ lại văn bản bên trong
    html = re.sub(r"<br\s*/?>", "\n", html, flags=re.IGNORECASE) # chuyển ký tự xuống dòng thành /n
    html = re.sub(r"</?p>", "", html, flags=re.IGNORECASE) # loại bỏ <p> trong html
    return html.strip() # trả hàm loại bỏ khoảng trắng đầu và cuối

def process_poem_content(html, poem_src, poem_url, default_title=""):
    cleaned = clean_poem_html(html) # làm sạch thẻ html
    pattern = re.compile(r"<b>(.*?)</b>\s*\n{2,}", flags=re.IGNORECASE)
    matches = list(pattern.finditer(cleaned))

    poems = []
    if matches:
        for i, match in enumerate(matches):
            title = match.group(1).strip()
            start = match.end()
            end = matches[i+1].start() if i + 1 < len(matches) else len(cleaned)
            content = cleaned[start:end].strip("\n")
            poems.append({
                "title": title,
                "content": content,
                "source": poem_src,
                "url": poem_url
            })
    else:
        poems.append({
            "title": default_title,
            "content": cleaned,
            "source": poem_src,
            "url": poem_url
        })

    return poems


In [ ]:
def scrape_poem(driver, poem_url):
    driver.get(poem_url) # từ link dữ liệu
    time.sleep(random.uniform(3, 5)) # dừng khoảng 3-5 giây để load dữ liệu

    # đợi tối đã 10s, Nếu phần tử được tìm thấy và hiển thị trên trang trong vòng 10 giây,
    # Selenium sẽ trả về đối tượng WebElement và gán cho biến poem_content_tag.
    # Nếu không tìm thấy (hoặc không visible) trong 10 giây,
    # Selenium sẽ ném ra một ngoại lệ (TimeoutException)
    poem_content_tag = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.poem-content"))
    )

    # lấy các content bên trong thuộc tính "poem_content_tag"
    html_content = poem_content_tag.get_attribute("innerHTML")


    # lấy thông tin nguồn tham khảo
    try:
        poem_src_tag = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='small']"))
        )
        poem_src = poem_src_tag.text
    except Exception as e:
        poem_src = ""

    return process_poem_content(html_content, poem_src, poem_url)

def scrape_poems(driver, num_pages=10):
    datasets = []
    for page_idx in range(1, num_pages + 1):
        poem_links = extract_poem_links(driver, page_idx) # đọc link dữ liệu
        for poem in poem_links:
            poem_url = poem["url"] # lấy link của từng dữ liệu
            try:
                poems = scrape_poem(driver, poem_url) # xử lý link để lấy dữ liệu
                datasets.extend(poems) # dữ liệu được lưu
            except Exception as e:
                print(f"Error processing {poem_url}: {e}")
                continue

    return datasets


In [ ]:
datasets = scrape_poems (driver , num_pages =10)
driver.quit ()

Error processing https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/V%E1%BB%9Bi-nh%E1%BB%AFng-g%C3%AC-%C4%91ang-c%C3%B3/poem-jImOkuDLzCJyRedC-trJ-Q: Message: 
Stacktrace:
#0 0x5c916de0c7f9 <unknown>
#1 0x5c916ddac3b3 <unknown>
#2 0x5c916daf4016 <unknown>
#3 0x5c916db2881e <unknown>
#4 0x5c916db5e8fb <unknown>
#5 0x5c916db4bded <unknown>
#6 0x5c916db5c9e1 <unknown>
#7 0x5c916db4bc93 <unknown>
#8 0x5c916db1dce4 <unknown>
#9 0x5c916db1f4d2 <unknown>
#10 0x5c916ddd8542 <unknown>
#11 0x5c916dde7ce7 <unknown>
#12 0x5c916dde79e4 <unknown>
#13 0x5c916ddec13a <unknown>
#14 0x5c916dde85b9 <unknown>
#15 0x5c916ddcde00 <unknown>
#16 0x5c916ddff5d2 <unknown>
#17 0x5c916ddff778 <unknown>
#18 0x5c916de17a1f <unknown>
#19 0x7b0d1323fac3 <unknown>
#20 0x7b0d132d1850 <unknown>

Error processing https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/V%C6%B0%E1%BB%A3t-tr%E1%BB%9F-ng%E1%BA%A1i-l%C3%A0-kh%C3%B3/poem-0HMLBkZ0f1EJMprmg1Mqew: Message: 
Stacktrace:
#0 0x5c916de0c7f9 <unknown>
#1 0x5c916ddac3b3 <unk

In [ ]:
# prompt: đọc file /content/poem_dataset.csv

import pandas as pd

# Assuming the file is in the /content directory
try:
  df = pd.read_csv('/content/poem_dataset.csv')
  print(df.head()) # Print the first few rows to verify
except FileNotFoundError:
  print("Error: File not found. Please make sure '/content/poem_dataset.csv' exists.")
except pd.errors.EmptyDataError:
  print("Error: The CSV file is empty.")
except pd.errors.ParserError:
  print("Error: Could not parse the CSV file. Please check the file format.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


   Unnamed: 0 title                                            content  \
0           0   NaN  Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...   
1           1   NaN  Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...   
2           2   NaN  Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...   
3           3   NaN  Chơi thân không có nghĩa\nKhông cãi nhau bao g...   
4           4   NaN  Có thể buồn chút ít\nMột mình, không người yêu...   

                                     source  \
0  [Thông tin 1 nguồn tham khảo đã được ẩn]   
1  [Thông tin 1 nguồn tham khảo đã được ẩn]   
2  [Thông tin 1 nguồn tham khảo đã được ẩn]   
3  [Thông tin 1 nguồn tham khảo đã được ẩn]   
4  [Thông tin 1 nguồn tham khảo đã được ẩn]   

                                                 url  
0  https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...  
1  https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...  
2  https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...  
3  https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...

In [ ]:
df = pd.DataFrame(df)
df.to_csv("poem_dataset.csv",index=True)
df

,Unnamed: 0,title,content,source,url
0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,NaN,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,NaN,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
102,102,NaN,Chợ trưa ai đợi mẹ\nBến khuya ai đợi đò\nSông ...,NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
103,103,NaN,Người ấy đến rất say\ntừ cao trời bay xuống\nm...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
104,104,NaN,Tôi giật mình đánh rơi\nCon mèo con xuống giến...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
105,105,NaN,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


### xây dựng mô hình huấn luyện

In [ ]:
!pip install -qq datasets evaluate transformers sentencepiece
!pip install -qq accelerate
!apt install git-lfs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset


In [ ]:
DATASET_PATH = "poem_dataset.csv"
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0.1,Unnamed: 0,title,content,source,url
0,0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,3,NaN,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,4,NaN,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...,...
102,102,102,NaN,Chợ trưa ai đợi mẹ\nBến khuya ai đợi đò\nSông ...,NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
103,103,103,NaN,Người ấy đến rất say\ntừ cao trời bay xuống\nm...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
104,104,104,NaN,Tôi giật mình đánh rơi\nCon mèo con xuống giến...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
105,105,105,NaN,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


In [ ]:
def split_content(content):
    samples = []
    poem_parts = content.split("\n\n")
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split("\n")
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)
    return samples

In [ ]:
df["content"] = df["content"].apply(lambda x: split_content(x))
df

,Unnamed: 0.1,Unnamed: 0,title,content,source,url
0,0,0,NaN,"[[Bạn xấu như chiếc bóng, Cứ bám riết theo anh...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,1,NaN,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,2,NaN,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,3,NaN,"[[Chơi thân không có nghĩa, Không cãi nhau bao...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,4,NaN,"[[Có thể buồn chút ít, Một mình, không người y...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...,...
102,102,102,NaN,"[[Chợ trưa ai đợi mẹ, Bến khuya ai đợi đò, Sôn...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
103,103,103,NaN,"[[Người ấy đến rất say, từ cao trời bay xuống,...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
104,104,104,NaN,[],[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
105,105,105,NaN,"[[Ôi, con sóng chết khô,, vật vờ trong bùn quá...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...


In [ ]:
df["content"][1]

[['Cái làm ta hạnh phúc',
  'Thực ra cũng chẳng nhiều',
  'Chỉ cần có ai đó',
  'Để ta thầm thương yêu'],
 ['Rồi thêm chút công việc',
  'Cho ta làm hàng ngày',
  'Cuối cùng, chút mơ mộng',
  'Để đưa ta lên mây']]

In [ ]:
df_exploded = df.explode("content")
print(df_exploded["content"][1])
df_exploded.reset_index(drop=True, inplace=True)
print(df_exploded["content"][1])
df_exploded = df_exploded.dropna(subset=["content"])
print(df_exploded["content"][1])
df_exploded


1    [Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...
1    [Rồi thêm chút công việc, Cho ta làm hàng ngày...
Name: content, dtype: object
['Nhưng họ sẽ biến mất', 'Khi trời phủ mây đen', 'Tức là khi anh đói', 'Trong túi không có tiền']
['Nhưng họ sẽ biến mất', 'Khi trời phủ mây đen', 'Tức là khi anh đói', 'Trong túi không có tiền']


,Unnamed: 0.1,Unnamed: 0,title,content,source,url
0,0,0,NaN,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,0,NaN,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,1,NaN,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,1,NaN,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,2,NaN,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...,...
210,103,103,NaN,"[người ấy đến rất dại, từ sâu rừng chạy ra, mộ...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
211,103,103,NaN,"[người ấy đến rất nhớ, từ xa biển dạt về, một ...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
212,103,103,NaN,"[người ấy đến rất thương, từ đất đen đứng dậy,...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
213,103,103,NaN,"[người ấy đến rất thơ, cạn cùng tôi ly nữa, tr...",Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...


In [ ]:
df_exploded["content"] = df_exploded["content"].apply(lambda x: "\n".join(x))
df_exploded


,Unnamed: 0.1,Unnamed: 0,title,content,source,url
0,0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,0,NaN,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,1,NaN,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...,...,...
210,103,103,NaN,người ấy đến rất dại\ntừ sâu rừng chạy ra\nmột...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
211,103,103,NaN,người ấy đến rất nhớ\ntừ xa biển dạt về\nmột n...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
212,103,103,NaN,người ấy đến rất thương\ntừ đất đen đứng dậy\n...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...
213,103,103,NaN,người ấy đến rất thơ\ncạn cùng tôi ly nữa\ntro...,Thu 84,https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%...


In [ ]:
# chuyể đổi dữ liệu từ dạng pandas sang huggingface
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)


In [ ]:
from huggingface_hub import notebook_login, whoami


In [ ]:
# tiền xử lý dữ liệu

MODEL_NAME = "gpt2"

notebook_login()


tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

# Thiết lập token PAD (do GPT2 không có sẵn token PAD)
tokenizer.pad_token = tokenizer.eos_token

MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
        row["content"],
        max_length=MAX_SEQ_LEN,
        padding="max_length",
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset["train"].column_names
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False )

In [ ]:
# BƯỚC 4: Cấu hình tham số huấn luyện
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
training_args = TrainingArguments(
    output_dir="gpt2_vi_poem_generation",
    save_strategy="epoch",       # Lưu checkpoint mỗi epoch
    learning_rate=2e-5,
    num_train_epochs=10,          # Tùy chỉnh số epoch
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Nếu có GPU hỗ trợ FP16
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=10,
    evaluation_strategy="epoch"
)

# BƯỚC 5: Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer = tokenizer
)

# BƯỚC 6: Bắt đầu huấn luyện
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-c39842a1b423>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nguyenanhtienbk1996 (nguyenanhtienbk1996-AAA) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.749700,2.719820
2,2.492000,2.523108
3,2.439500,2.426164
4,2.342500,2.385770
5,2.270300,2.333569
6,2.202400,2.317105
7,2.055900,2.301759
8,2.045100,2.294203
9,2.025900,2.291813
10,2.017300,2.285939


TrainOutput(global_step=870, training_loss=2.2967817481906936, metrics={'train_runtime': 449.245, 'train_samples_per_second': 3.873, 'train_steps_per_second': 1.937, 'total_flos': 88798464000000.0, 'train_loss': 2.2967817481906936, 'epoch': 10.0})

In [ ]:

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Check for CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"
#model.to(device)

# Example prompt
prompt = "Học học nữa học mãi\n"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate text
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)

# Decode the generated text
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]

# Print the generated text line by line
print()
for line in results.split("\n"):
  print(line)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Màt dư chính khiang láng còn ngây...


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "tienabcd/gpt2-viviki"  # Tên repo bạn muốn
api.create_repo(repo_id=repo_id, exist_ok=True)  # exist_ok=True để tránh lỗi nếu đã tồn tại


RepoUrl('https://huggingface.co/tienabcd/gpt2-viviki', endpoint='https://huggingface.co', repo_type='model', repo_id='tienabcd/gpt2-viviki')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Replace with the actual path to your saved checkpoint
model_checkpoint = "/content/gpt2_vi_poem_generation/checkpoint-870"  # 'xxx' is the checkpoint number (e.g., checkpoint-1000)

model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)

# Đặt repo_id theo tên model repo của bạn
repo_id = "tienabcd/gpt2-viviki"  # Update with your desired repo ID

# Push the model and tokenizer to the Hub
model.push_to_hub(repo_id=repo_id)
tokenizer.push_to_hub(repo_id=repo_id)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tienabcd/gpt2-viviki/commit/9d2531ebd6524b3526a3285aa834a7f0bd1eb00b', commit_message='Upload tokenizer', commit_description='', oid='9d2531ebd6524b3526a3285aa834a7f0bd1eb00b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tienabcd/gpt2-viviki', endpoint='https://huggingface.co', repo_type='model', repo_id='tienabcd/gpt2-viviki'), pr_revision=None, pr_num=None)

## test code HTML

In [ ]:
link_1 = "https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/B%E1%BA%A1n-x%E1%BA%A5u-nh%C6%B0-chi%E1%BA%BFc-b%C3%B3ng/poem-4r1MApTHwgzgxicR8oy7nQ"
driver.get(link_1) # từ link dữ liệu
time.sleep(random.uniform(3, 5)) # dừng khoảng 3-5 giây để load dữ liệu

# đợi tối đã 10s, Nếu phần tử được tìm thấy và hiển thị trên trang trong vòng 10 giây,
# Selenium sẽ trả về đối tượng WebElement và gán cho biến poem_content_tag.
# Nếu không tìm thấy (hoặc không visible) trong 10 giây,
# Selenium sẽ ném ra một ngoại lệ (TimeoutException)
poem_content_tag = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.poem-content")))
poem_content_tag

<selenium.webdriver.remote.webelement.WebElement (session="be6470626321e05cbff6cdebe0324147", element="c99a6151-bf5b-4a49-ae39-b49ad601571f")>

In [ ]:
html = poem_content_tag.get_attribute("innerHTML")
html

'<img class="author-poster-cert pull-right" src="/image/icon_cert_silver.svg" data-toggle="tooltip" title="" data-original-title="Bài thơ này được căn cứ theo thông tin có sự xác nhận của tác giả, hoặc được tham khảo từ ấn phẩm hay nguồn tin do tác giả trực tiếp công bố"><p>Bạn xấu như chiếc bóng<br>Cứ bám riết theo anh<br>Khi anh sáng, rực rỡ<br>Như mặt trời long lanh<br><br>Nhưng họ sẽ biến mất<br>Khi trời phủ mây đen<br>Tức là khi anh đói<br>Trong túi không có tiền</p>'

In [ ]:
cleaned = clean_poem_html(html)
cleaned

'Bạn xấu như chiếc bóng\nCứ bám riết theo anh\nKhi anh sáng, rực rỡ\nNhư mặt trời long lanh\n\nNhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTức là khi anh đói\nTrong túi không có tiền'

In [ ]:
pattern = re.compile(r"<b>(.*?)</b>\s*\n{2,}", flags=re.IGNORECASE)
matches = list(pattern.finditer(cleaned))
matches

[]

In [ ]:
default_title=""
poem_src = ""
poems = []
if matches:
    for i, match in enumerate(matches):
        title = match.group(1).strip()
        start = match.end()
        print("start:",start)
        end = matches[i+1].start() if i + 1 < len(matches) else len(cleaned)
        print("end:",start)
        content = cleaned[start:end].strip("\n")
        poems.append({
            "title": title,
            "content": content,
            "source": poem_src,
            "url": link_1
        })
else:
    poems.append({
        "title": default_title,
        "content": cleaned,
        "source": poem_src,
        "url": link_1
    })
print(poems)

[{'title': '', 'content': 'Bạn xấu như chiếc bóng\nCứ bám riết theo anh\nKhi anh sáng, rực rỡ\nNhư mặt trời long lanh\n\nNhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTức là khi anh đói\nTrong túi không có tiền', 'source': '', 'url': 'https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/B%E1%BA%A1n-x%E1%BA%A5u-nh%C6%B0-chi%E1%BA%BFc-b%C3%B3ng/poem-4r1MApTHwgzgxicR8oy7nQ'}]


### test hàm làm sạch html "clean_poem_html"

In [ ]:
html = poem_content_tag.get_attribute("innerHTML")
html

In [ ]:
# hàm clean html
html = re.sub(r"<img.*?>", "", html, flags=re.IGNORECASE | re.DOTALL) # xóa các thẻ ảnh
html

In [ ]:
html = re.sub(r"<i>.*?</i>", "", html, flags=re.IGNORECASE | re.DOTALL) # loại bỏ các thẻ in nghiêng
html

In [ ]:
html = re.sub(r"<b>(.*?)</b>(?!\s*(?:<br\s*/?>\s*){2,})",
              r"\1",
              html,
              flags=re.IGNORECASE) # loại bỏ các thẻ in đậm nhưng giữ lại văn bản bên trong
html

In [ ]:
html = re.sub(r"<br\s*/?>", "\n", html, flags=re.IGNORECASE) # chuyển ký tự xuống dòng thành /n
html

In [ ]:
html = re.sub(r"</?p>", "", html, flags=re.IGNORECASE)
html

In [ ]:
html.strip()

#### ví dụ cho strip()

In [ ]:

html_1 = "  Với trẻ con, tốt nhất\nLà không cho chúng tiền\nGiúp làm bài, đi dạo\nĐỡ tốn và đỡ phiền\n\nThật đẹp và ý nghĩa\nBố dắt con dạo chơi\nVừa đi vừa bày dạy\nĐể giúp con thành người  "
html_1

In [ ]:

html_1.strip()